<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Maratona_BehindTheCode_IBM21/blob/main/desafio-1/MBTC2021_df1_TotalBan_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTC2021_df1-TotalBan_pt2

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score

## Get Datas

In [ ]:
accounts = pd.read_csv(
    "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ACCOUNTS.csv")
demograph = pd.read_csv(
    "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/DEMOGRAPHICS.csv")
loans = pd.read_csv(
    "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/LOANS.csv")
answer = pd.read_csv(
    "https://raw.githubusercontent.com/maratonadev/desafio-1-2021/main/assets/data/ANSWERS.csv")

In [ ]:
join = accounts.sort_values("ID").set_index("ID").join(
    [demograph.set_index("ID"), loans.set_index("ID")],
    how="outer")

In [ ]:
answer

,ID,CHECKING_BALANCE,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EXISTING_SAVINGS,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,ALLOW
0,1078506,25.84,465,NO_CREDITS,CAR_NEW,250,38.35,0,1,F,NONE,2,REAL_ESTATE,32,STORES,OWN,1,3,1,0,1,TO_FILL
1,947575,-439.72,120,ALL_CREDITS_PAID_BACK,CAR_NEW,250,1.62,0,2,M,CO-APPLICANT,2,REAL_ESTATE,25,STORES,RENT,1,3,1,0,1,TO_FILL
2,133043,169.45,527,PRIOR_PAYMENTS_DELAYED,RADIO_TV,2207,184.97,6,2,M,NONE,3,SAVINGS_INSURANCE,32,STORES,OWN,2,1,2,0,1,TO_FILL
3,1031383,NO_CHECKING,720,OUTSTANDING_CREDIT,CAR_USED,5064,731.03,6,4,M,NONE,2,CAR_OTHER,49,NONE,OWN,2,3,1,0,1,TO_FILL
4,491563,NO_CHECKING,420,OUTSTANDING_CREDIT,FURNITURE,3763,396.26,4,3,F,NONE,3,CAR_OTHER,46,STORES,OWN,1,1,2,1,1,TO_FILL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1023481,NO_CHECKING,1116,PRIOR_PAYMENTS_DELAYED,APPLIANCES,3960,1224.22,8,4,F,NONE,3,SAVINGS_INSURANCE,46,NONE,OWN,2,2,1,0,0,TO_FILL
996,1211101,-399.5,720,ALL_CREDITS_PAID_BACK,CAR_NEW,1834,57.44,7,3,M,NONE,1,SAVINGS_INSURANCE,27,NONE,OWN,1,3,1,0,1,TO_FILL
997,599972,158.3,558,CREDITS_PAID_TO_DATE,CAR_NEW,2348,520.17,1,3,M,CO-APPLICANT,2,SAVINGS_INSURANCE,42,STORES,OWN,1,1,1,0,1,TO_FILL
998,1146844,NO_CHECKING,1170,PRIOR_PAYMENTS_DELAYED,FURNITURE,5457,531.02,4,3,M,CO-APPLICANT,3,CAR_OTHER,42,NONE,OWN,1,3,1,1,1,TO_FILL


## Drop Columns

In [ ]:
unwanted_columns = ["ALLOW", "ID", "CHECKING_BALANCE", "EXISTING_SAVINGS"]
df1 = join.drop(list(set(unwanted_columns) - set(["ID", "ALLOW"])), axis=1)
df2 = answer.drop(unwanted_columns, axis=1)

## Changing Columns

In [ ]:
df1.columns

Index(['CREDIT_HISTORY', 'EXISTING_CREDITS_COUNT', 'SEX', 'AGE', 'JOB_TYPE',
       'DEPENDENTS', 'TELEPHONE', 'FOREIGN_WORKER', 'EMPLOYMENT_DURATION',
       'PROPERTY', 'HOUSING', 'CURRENT_RESIDENCE_DURATION', 'PAYMENT_TERM',
       'INSTALLMENT_PLANS', 'INSTALLMENT_PERCENT', 'LOAN_PURPOSE',
       'LOAN_AMOUNT', 'OTHERS_ON_LOAN', 'ALLOW'],
      dtype='object')

In [ ]:
df2.columns

Index(['PAYMENT_TERM', 'CREDIT_HISTORY', 'LOAN_PURPOSE', 'LOAN_AMOUNT',
       'EMPLOYMENT_DURATION', 'INSTALLMENT_PERCENT', 'SEX', 'OTHERS_ON_LOAN',
       'CURRENT_RESIDENCE_DURATION', 'PROPERTY', 'AGE', 'INSTALLMENT_PLANS',
       'HOUSING', 'EXISTING_CREDITS_COUNT', 'JOB_TYPE', 'DEPENDENTS',
       'TELEPHONE', 'FOREIGN_WORKER'],
      dtype='object')

In [ ]:
cols_df1 = df1.columns
cols_df2 = df2.columns
cols_df1 = cols_df2
df1 = df1[cols_df1]
df1["ALLOW"] = join["ALLOW"]

In [ ]:
df1

,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,ALLOW
ID,,,,,,,,,,,,,,,,,,,
98,961.0,CREDITS_PAID_TO_DATE,OTHER,1889.0,0.0,3.0,F,NONE,3.0,SAVINGS_INSURANCE,32.0,NONE,OWN,1.0,3.0,1.0,0.0,1.0,1
317,540.0,CREDITS_PAID_TO_DATE,CAR_NEW,462.0,1.0,2.0,F,NONE,2.0,SAVINGS_INSURANCE,37.0,STORES,OWN,2.0,3.0,1.0,0.0,1.0,1
407,465.0,PRIOR_PAYMENTS_DELAYED,FURNITURE,250.0,3.0,2.0,M,NONE,3.0,REAL_ESTATE,28.0,NONE,OWN,2.0,3.0,1.0,1.0,0.0,1
2375,480.0,NaN,VACATION,3109.0,6.0,3.0,F,NONE,1.0,CAR_OTHER,36.0,NONE,OWN,NaN,3.0,1.0,0.0,1.0,1
2731,330.0,NaN,CAR_NEW,4553.0,0.0,3.0,F,NONE,3.0,SAVINGS_INSURANCE,22.0,NONE,OWN,NaN,2.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319170,810.0,CREDITS_PAID_TO_DATE,FURNITURE,4650.0,1.0,3.0,M,NONE,4.0,SAVINGS_INSURANCE,40.0,NONE,OWN,1.0,3.0,1.0,0.0,1.0,1
1319397,330.0,PRIOR_PAYMENTS_DELAYED,FURNITURE,250.0,7.0,3.0,M,NONE,3.0,CAR_OTHER,32.0,BANK,OWN,1.0,0.0,1.0,0.0,1.0,1
1319835,992.0,NaN,APPLIANCES,6536.0,11.0,5.0,M,CO-APPLICANT,5.0,UNKNOWN,54.0,STORES,OWN,NaN,1.0,2.0,1.0,1.0,0


## Changing Types

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 98 to 1320082
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PAYMENT_TERM                3975 non-null   float64
 1   CREDIT_HISTORY              2462 non-null   object 
 2   LOAN_PURPOSE                3979 non-null   object 
 3   LOAN_AMOUNT                 3981 non-null   float64
 4   EMPLOYMENT_DURATION         3923 non-null   float64
 5   INSTALLMENT_PERCENT         3974 non-null   float64
 6   SEX                         3920 non-null   object 
 7   OTHERS_ON_LOAN              3977 non-null   object 
 8   CURRENT_RESIDENCE_DURATION  3917 non-null   float64
 9   PROPERTY                    3918 non-null   object 
 10  AGE                         3918 non-null   float64
 11  INSTALLMENT_PLANS           3978 non-null   object 
 12  HOUSING                     3912 non-null   object 
 13  EXISTING_CREDITS_COUNT      2

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   PAYMENT_TERM                1000 non-null   int64 
 1   CREDIT_HISTORY              1000 non-null   object
 2   LOAN_PURPOSE                1000 non-null   object
 3   LOAN_AMOUNT                 1000 non-null   int64 
 4   EMPLOYMENT_DURATION         1000 non-null   int64 
 5   INSTALLMENT_PERCENT         1000 non-null   int64 
 6   SEX                         1000 non-null   object
 7   OTHERS_ON_LOAN              1000 non-null   object
 8   CURRENT_RESIDENCE_DURATION  1000 non-null   int64 
 9   PROPERTY                    1000 non-null   object
 10  AGE                         1000 non-null   int64 
 11  INSTALLMENT_PLANS           1000 non-null   object
 12  HOUSING                     1000 non-null   object
 13  EXISTING_CREDITS_COUNT      1000 non-null   int64

In [ ]:
df1 = df1.dropna()

In [ ]:
floats = df1.select_dtypes(include=['float']).columns
df1[floats] = df1[floats].astype(int)

## Preprocessing

In [ ]:
X = df1.drop("ALLOW", axis=1)
y = df1["ALLOW"]

In [ ]:
enc = OrdinalEncoder()

objs = np.array(X.select_dtypes("object"))
enc.fit(objs)
objs = enc.transform(objs)
X[X.select_dtypes("object").columns] = objs
X

,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER
ID,,,,,,,,,,,,,,,,,,
98,961,1.0,6.0,1889,0,3,0.0,2.0,3,2.0,32,1.0,1.0,1,3,1,0,1
317,540,1.0,2.0,462,1,2,0.0,2.0,2,2.0,37,2.0,1.0,2,3,1,0,1
407,465,4.0,5.0,250,3,2,1.0,2.0,3,1.0,28,1.0,1.0,2,3,1,1,0
2893,1050,3.0,0.0,7138,10,5,1.0,0.0,4,3.0,49,1.0,0.0,2,3,2,1,1
3412,155,0.0,2.0,1523,0,2,0.0,2.0,2,1.0,19,1.0,2.0,1,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317323,660,2.0,5.0,1822,3,3,0.0,2.0,2,2.0,32,1.0,1.0,1,3,1,0,1
1318378,450,1.0,3.0,4097,3,3,1.0,2.0,3,0.0,43,1.0,1.0,2,3,1,0,1
1319170,810,1.0,5.0,4650,1,3,1.0,2.0,4,2.0,40,1.0,1.0,1,3,1,0,1


In [ ]:
objs_df2 = np.array(df2.select_dtypes("object"))
enc.fit(objs_df2)
objs_df2 = enc.transform(objs_df2)
df2[df2.select_dtypes("object").columns] = objs_df2
df2

,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER
0,465,2.0,2.0,250,0,1,0.0,2.0,2,1.0,32,2.0,1.0,1,3,1,0,1
1,120,0.0,2.0,250,0,2,1.0,0.0,2,1.0,25,2.0,2.0,1,3,1,0,1
2,527,4.0,7.0,2207,6,2,1.0,2.0,3,2.0,32,2.0,1.0,2,1,2,0,1
3,720,3.0,3.0,5064,6,4,1.0,2.0,2,0.0,49,1.0,1.0,2,3,1,0,1
4,420,3.0,5.0,3763,4,3,0.0,2.0,3,0.0,46,2.0,1.0,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1116,4.0,0.0,3960,8,4,0.0,2.0,3,2.0,46,1.0,1.0,2,2,1,0,0
996,720,0.0,2.0,1834,7,3,1.0,2.0,1,2.0,27,1.0,1.0,1,3,1,0,1
997,558,1.0,2.0,2348,1,3,1.0,0.0,2,2.0,42,2.0,1.0,1,1,1,0,1
998,1170,4.0,5.0,5457,4,3,1.0,0.0,3,0.0,42,1.0,1.0,1,3,1,1,1


## Modeling

In [ ]:
model = GradientBoostingClassifier(
    learning_rate=0.01, max_depth=5, max_features=0.25, min_samples_leaf=11,
    min_samples_split=20, n_estimators=100, subsample=1.0, random_state=42
)
model.fit(X, y)
allow = model.predict(df2)

In [ ]:
answer["ALLOW"] = allow
answer

,ID,CHECKING_BALANCE,PAYMENT_TERM,CREDIT_HISTORY,LOAN_PURPOSE,LOAN_AMOUNT,EXISTING_SAVINGS,EMPLOYMENT_DURATION,INSTALLMENT_PERCENT,SEX,OTHERS_ON_LOAN,CURRENT_RESIDENCE_DURATION,PROPERTY,AGE,INSTALLMENT_PLANS,HOUSING,EXISTING_CREDITS_COUNT,JOB_TYPE,DEPENDENTS,TELEPHONE,FOREIGN_WORKER,ALLOW
0,1078506,25.84,465,NO_CREDITS,CAR_NEW,250,38.35,0,1,F,NONE,2,REAL_ESTATE,32,STORES,OWN,1,3,1,0,1,1
1,947575,-439.72,120,ALL_CREDITS_PAID_BACK,CAR_NEW,250,1.62,0,2,M,CO-APPLICANT,2,REAL_ESTATE,25,STORES,RENT,1,3,1,0,1,1
2,133043,169.45,527,PRIOR_PAYMENTS_DELAYED,RADIO_TV,2207,184.97,6,2,M,NONE,3,SAVINGS_INSURANCE,32,STORES,OWN,2,1,2,0,1,1
3,1031383,NO_CHECKING,720,OUTSTANDING_CREDIT,CAR_USED,5064,731.03,6,4,M,NONE,2,CAR_OTHER,49,NONE,OWN,2,3,1,0,1,1
4,491563,NO_CHECKING,420,OUTSTANDING_CREDIT,FURNITURE,3763,396.26,4,3,F,NONE,3,CAR_OTHER,46,STORES,OWN,1,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1023481,NO_CHECKING,1116,PRIOR_PAYMENTS_DELAYED,APPLIANCES,3960,1224.22,8,4,F,NONE,3,SAVINGS_INSURANCE,46,NONE,OWN,2,2,1,0,0,1
996,1211101,-399.5,720,ALL_CREDITS_PAID_BACK,CAR_NEW,1834,57.44,7,3,M,NONE,1,SAVINGS_INSURANCE,27,NONE,OWN,1,3,1,0,1,1
997,599972,158.3,558,CREDITS_PAID_TO_DATE,CAR_NEW,2348,520.17,1,3,M,CO-APPLICANT,2,SAVINGS_INSURANCE,42,STORES,OWN,1,1,1,0,1,1
998,1146844,NO_CHECKING,1170,PRIOR_PAYMENTS_DELAYED,FURNITURE,5457,531.02,4,3,M,CO-APPLICANT,3,CAR_OTHER,42,NONE,OWN,1,3,1,1,1,1


## Submiting

In [ ]:
answer.to_csv("ANSWERS.csv", index=False)

In [ ]:
answer["ALLOW"].value_counts()

1    820
0    180
Name: ALLOW, dtype: int64

In [ ]:
y.value_counts()

1    1481
0     776
Name: ALLOW, dtype: int64